In [ ]:
#https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from wordcloud import WordCloud, STOPWORDS

In [4]:
import nltk

In [5]:
xl = pd.ExcelFile('Nest_US_Video_Verbatims_Q2-Q4.xlsx')

In [6]:
xl.sheet_names

['Q2 2019 - 19Q2_015 (1)',
 'Q2 2019 - 19Q2_015 (2)',
 'Q3 2019 - 19Q2_016',
 'Q3 2019 - 12Q3_011',
 'Q4 2019 - 19Q3_076',
 'Q4 2019 - 19Q4_011',
 'Q4 2019 - 19Q4_026',
 'Q4 2019 - 19Q4_044',
 'Q4 2019 - 19Q4_059']

In [7]:
Q2_wave1 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q2 2019 - 19Q2_015 (1)")
Q2_wave2 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q2 2019 - 19Q2_015 (2)")
Q3_wave1 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q3 2019 - 19Q2_016")
Q3_wave2 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q3 2019 - 12Q3_011")
Q4_wave1 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q4 2019 - 19Q3_076")
Q4_wave2 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q4 2019 - 19Q4_026")
Q4_wave3 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q4 2019 - 19Q4_044")
Q4_wave4 = pd.read_excel('Nest_US_Video_Verbatims_Q2-Q4.xlsx', sheet_name="Q4 2019 - 19Q4_059")

In [8]:
Q4_wave1.head()

,Creative,Q189,Q190,Q191
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet,It was simple and short,It didn’t give that much information
1,Test 4: Max YT V1 :06s,Smart technology in your home for safety and l...,Current,Might e too expenisve for me to huy
2,Test 5: Wedding Photo V1 :06s,Not sure,Everything,Nothing
3,Control,NaN,NaN,NaN
4,Test 7: Internet You Can Talk To V1 :06s,Not into it,Not into it,Not into it


In [9]:
Q4_wave1.rename(columns = {'Q189':'Takeaways'}, inplace = True)
Q4_wave1.rename(columns = {'Q190':'Likes'}, inplace = True)
Q4_wave1.rename(columns = {'Q191':'Dislikes'}, inplace = True) 

In [10]:
Q4_wave1.head()

,Creative,Takeaways,Likes,Dislikes
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet,It was simple and short,It didn’t give that much information
1,Test 4: Max YT V1 :06s,Smart technology in your home for safety and l...,Current,Might e too expenisve for me to huy
2,Test 5: Wedding Photo V1 :06s,Not sure,Everything,Nothing
3,Control,NaN,NaN,NaN
4,Test 7: Internet You Can Talk To V1 :06s,Not into it,Not into it,Not into it


In [11]:
test = Q4_wave1[['Creative', 'Takeaways']]
print(test.shape)
test.head()

(2250, 2)


,Creative,Takeaways
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet
1,Test 4: Max YT V1 :06s,Smart technology in your home for safety and l...
2,Test 5: Wedding Photo V1 :06s,Not sure
3,Control,NaN
4,Test 7: Internet You Can Talk To V1 :06s,Not into it


In [12]:
#drops control cells and blank takeaways
test1 = test.dropna()
print(test1.shape)
test1.head()

(1838, 2)


,Creative,Takeaways
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet
1,Test 4: Max YT V1 :06s,Smart technology in your home for safety and l...
2,Test 5: Wedding Photo V1 :06s,Not sure
4,Test 7: Internet You Can Talk To V1 :06s,Not into it
5,Test 2: Baby Photo V3 :06s,Shows images of moments


In [13]:
# the bigger table above broken up into smaller tables based on each creative and stored in a dictionary where the key is the creative name
test1_group = dict(tuple(test1.groupby('Creative')))

In [14]:
listcreativetest = test1.Creative.unique()
list(listcreativetest)

['Test 7: Internet You Can Talk To V1 :06s',
 'Test 4: Max YT V1 :06s',
 'Test 5: Wedding Photo V1 :06s',
 'Test 2: Baby Photo V3 :06s',
 'Test 1: Baby Photo V1 :06s',
 'Test 3: Halloween Photo V1 :06s',
 'Test 8: Router V1 :06s',
 'Test 6: Wedding Photo V2 :06s']

In [15]:
test1_creative7 = test1_group['Test 7: Internet You Can Talk To V1 :06s']
test1_creative7.head()

,Creative,Takeaways
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet
4,Test 7: Internet You Can Talk To V1 :06s,Not into it
11,Test 7: Internet You Can Talk To V1 :06s,Apple
17,Test 7: Internet You Can Talk To V1 :06s,It's to make everyday life easier.
31,Test 7: Internet You Can Talk To V1 :06s,Showing me the product and how it works


In [16]:
test1_creative7

,Creative,Takeaways
0,Test 7: Internet You Can Talk To V1 :06s,That you can talk to the internet
4,Test 7: Internet You Can Talk To V1 :06s,Not into it
11,Test 7: Internet You Can Talk To V1 :06s,Apple
17,Test 7: Internet You Can Talk To V1 :06s,It's to make everyday life easier.
31,Test 7: Internet You Can Talk To V1 :06s,Showing me the product and how it works
35,Test 7: Internet You Can Talk To V1 :06s,Helps with everything
53,Test 7: Internet You Can Talk To V1 :06s,Helpful
69,Test 7: Internet You Can Talk To V1 :06s,To get you to purchase google nest
103,Test 7: Internet You Can Talk To V1 :06s,Nothing
104,Test 7: Internet You Can Talk To V1 :06s,I love it. Want to try it out. Would totally l...


In [17]:
takeaways = test1_creative7['Takeaways'].tolist()
takeaways

['That you can talk to the internet ',
 'Not into it',
 'Apple',
 "It's to make everyday life easier.",
 'Showing me the product and how it works ',
 'Helps with everything',
 'Helpful',
 'To get you to purchase google nest',
 'Nothing ',
 'I love it. Want to try it out. Would totally love one of my own. To connect to my Google home mini. To learn more about it. I really am into it. I want to tell my friends and family about it. ',
 'Voice ',
 'It shows that you can have a casual interaction and that the technology is here to help.',
 'None ',
 'Great Product and have smart technologies ',
 'Not sure',
 'That you have to say hey Google for it to activate',
 'It didn’t show enough ',
 'Google',
 'You can talk to it and ask it questions',
 'It’s small and fits in with your home decor',
 'If you have any questions you can just ask google nest ',
 'It has a sound bite that will stick in your mind the more you hear the ad.',
 'You can search google by talking to the Google Nest',
 'Not sure

In [18]:
#turning the contents into lowercase letters
takeaways = np.char.lower(takeaways)

In [19]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [20]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [21]:
stop_words = set(stopwords.words('english'))

In [22]:
word_tokens = []
for response in takeaways:
    word_tokens.append(word_tokenize(response))

In [54]:
word_tokens[:5]

[['that', 'you', 'can', 'talk', 'to', 'the', 'internet'],
 ['not', 'into', 'it'],
 ['apple'],
 ['it', "'s", 'to', 'make', 'everyday', 'life', 'easier', '.'],
 ['showing', 'me', 'the', 'product', 'and', 'how', 'it', 'works']]

In [24]:
word_tokens_list = [item for sublist in word_tokens for item in sublist]

In [25]:
word_tokens_list

['that',
 'you',
 'can',
 'talk',
 'to',
 'the',
 'internet',
 'not',
 'into',
 'it',
 'apple',
 'it',
 "'s",
 'to',
 'make',
 'everyday',
 'life',
 'easier',
 '.',
 'showing',
 'me',
 'the',
 'product',
 'and',
 'how',
 'it',
 'works',
 'helps',
 'with',
 'everything',
 'helpful',
 'to',
 'get',
 'you',
 'to',
 'purchase',
 'google',
 'nest',
 'nothing',
 'i',
 'love',
 'it',
 '.',
 'want',
 'to',
 'try',
 'it',
 'out',
 '.',
 'would',
 'totally',
 'love',
 'one',
 'of',
 'my',
 'own',
 '.',
 'to',
 'connect',
 'to',
 'my',
 'google',
 'home',
 'mini',
 '.',
 'to',
 'learn',
 'more',
 'about',
 'it',
 '.',
 'i',
 'really',
 'am',
 'into',
 'it',
 '.',
 'i',
 'want',
 'to',
 'tell',
 'my',
 'friends',
 'and',
 'family',
 'about',
 'it',
 '.',
 'voice',
 'it',
 'shows',
 'that',
 'you',
 'can',
 'have',
 'a',
 'casual',
 'interaction',
 'and',
 'that',
 'the',
 'technology',
 'is',
 'here',
 'to',
 'help',
 '.',
 'none',
 'great',
 'product',
 'and',
 'have',
 'smart',
 'technologies',


In [26]:
filtered_words = [w for w in word_tokens_list if not w in stop_words]

In [30]:
print(len(filtered_words))
filtered_words [:5]

802


['talk', 'internet', 'apple', "'s", 'make']

In [44]:
symbols = "!\"“'’#$%&()*+-./:;<=>?@[\]^_`{|}~\n0123456789"
for i in symbols:
    filtered_words = np.char.replace(filtered_words, i, "")

In [45]:
filtered_words

array(['talk', 'internet', 'apple', 's', 'make', 'everyday', 'life',
       'easier', '', 'showing', 'product', 'works', 'helps', 'everything',
       'helpful', 'get', 'purchase', 'google', 'nest', 'nothing', 'love',
       '', 'want', 'try', '', 'would', 'totally', 'love', 'one', '',
       'connect', 'google', 'home', 'mini', '', 'learn', '', 'really', '',
       'want', 'tell', 'friends', 'family', '', 'voice', 'shows',
       'casual', 'interaction', 'technology', 'help', '', 'none', 'great',
       'product', 'smart', 'technologies', 'sure', 'say', 'hey', 'google',
       'activate', '', 'show', 'enough', 'google', 'talk', 'ask',
       'questions', '', 'small', 'fits', 'home', 'decor', 'questions',
       'ask', 'google', 'nest', 'sound', 'bite', 'stick', 'mind', 'hear',
       'ad', '', 'search', 'google', 'talking', 'google', 'nest', 'sure',
       'fact', 'talk', 'communicate', 'device', 'get', 'response',
       'connect', 'internet', 'voice', '', 'okay', 'honey', 'cha', 'co

In [55]:
filtered_words

['talk',
 'internet',
 'apple',
 's',
 'make',
 'everyday',
 'life',
 'easier',
 'showing',
 'product',
 'works',
 'helps',
 'everything',
 'helpful',
 'get',
 'purchase',
 'google',
 'nest',
 'nothing',
 'love',
 'want',
 'try',
 'would',
 'totally',
 'love',
 'one',
 'connect',
 'google',
 'home',
 'mini',
 'learn',
 'really',
 'want',
 'tell',
 'friends',
 'family',
 'voice',
 'shows',
 'casual',
 'interaction',
 'technology',
 'help',
 'none',
 'great',
 'product',
 'smart',
 'technologies',
 'sure',
 'say',
 'hey',
 'google',
 'activate',
 'show',
 'enough',
 'google',
 'talk',
 'ask',
 'questions',
 'small',
 'fits',
 'home',
 'decor',
 'questions',
 'ask',
 'google',
 'nest',
 'sound',
 'bite',
 'stick',
 'mind',
 'hear',
 'ad',
 'search',
 'google',
 'talking',
 'google',
 'nest',
 'sure',
 'fact',
 'talk',
 'communicate',
 'device',
 'get',
 'response',
 'connect',
 'internet',
 'voice',
 'okay',
 'honey',
 'cha',
 'coo',
 'love',
 'love',
 'comfortable',
 'part',
 'home',
 'r

In [59]:
filtered_words = list(filter(None, filtered_words))
len(filtered_words)

705

In [ ]:
#Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, 
#and often includes the removal of derivational affixes. 
#Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, 
#normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma 

In [60]:
from nltk.stem import WordNetLemmatizer

In [63]:
lemmatizer = WordNetLemmatizer() 

In [64]:
lemma_words = []
for word in filtered_words:
    lemma_words.append(lemmatizer.lemmatize(word))

In [67]:
lemma_words

['talk',
 'internet',
 'apple',
 's',
 'make',
 'everyday',
 'life',
 'easier',
 'showing',
 'product',
 'work',
 'help',
 'everything',
 'helpful',
 'get',
 'purchase',
 'google',
 'nest',
 'nothing',
 'love',
 'want',
 'try',
 'would',
 'totally',
 'love',
 'one',
 'connect',
 'google',
 'home',
 'mini',
 'learn',
 'really',
 'want',
 'tell',
 'friend',
 'family',
 'voice',
 'show',
 'casual',
 'interaction',
 'technology',
 'help',
 'none',
 'great',
 'product',
 'smart',
 'technology',
 'sure',
 'say',
 'hey',
 'google',
 'activate',
 'show',
 'enough',
 'google',
 'talk',
 'ask',
 'question',
 'small',
 'fit',
 'home',
 'decor',
 'question',
 'ask',
 'google',
 'nest',
 'sound',
 'bite',
 'stick',
 'mind',
 'hear',
 'ad',
 'search',
 'google',
 'talking',
 'google',
 'nest',
 'sure',
 'fact',
 'talk',
 'communicate',
 'device',
 'get',
 'response',
 'connect',
 'internet',
 'voice',
 'okay',
 'honey',
 'cha',
 'coo',
 'love',
 'love',
 'comfortable',
 'part',
 'home',
 'relaxed',


In [40]:
from nltk.stem import PorterStemmer 

In [41]:
ps = PorterStemmer()

In [65]:
final_stemmed_words = []
for word in lemma_words:
    final_stemmed_words.append(ps.stem(word))

In [66]:
final_stemmed_words

['talk',
 'internet',
 'appl',
 's',
 'make',
 'everyday',
 'life',
 'easier',
 'show',
 'product',
 'work',
 'help',
 'everyth',
 'help',
 'get',
 'purchas',
 'googl',
 'nest',
 'noth',
 'love',
 'want',
 'tri',
 'would',
 'total',
 'love',
 'one',
 'connect',
 'googl',
 'home',
 'mini',
 'learn',
 'realli',
 'want',
 'tell',
 'friend',
 'famili',
 'voic',
 'show',
 'casual',
 'interact',
 'technolog',
 'help',
 'none',
 'great',
 'product',
 'smart',
 'technolog',
 'sure',
 'say',
 'hey',
 'googl',
 'activ',
 'show',
 'enough',
 'googl',
 'talk',
 'ask',
 'question',
 'small',
 'fit',
 'home',
 'decor',
 'question',
 'ask',
 'googl',
 'nest',
 'sound',
 'bite',
 'stick',
 'mind',
 'hear',
 'ad',
 'search',
 'googl',
 'talk',
 'googl',
 'nest',
 'sure',
 'fact',
 'talk',
 'commun',
 'devic',
 'get',
 'respons',
 'connect',
 'internet',
 'voic',
 'okay',
 'honey',
 'cha',
 'coo',
 'love',
 'love',
 'comfort',
 'part',
 'home',
 'relax',
 'easi',
 'probabl',
 'soon',
 'enough',
 'good',

In [57]:
for i in final_stemmed_words:
    if len(i) == 1:
        final_stemmed_words.remove(i)
len(final_stemmed_words)

693

In [58]:
final_stemmed_words

['talk',
 'internet',
 'appl',
 'make',
 'everyday',
 'life',
 'easier',
 'show',
 'product',
 'work',
 'help',
 'everyth',
 'help',
 'get',
 'purchas',
 'googl',
 'nest',
 'noth',
 'love',
 'want',
 'tri',
 'would',
 'total',
 'love',
 'one',
 'connect',
 'googl',
 'home',
 'mini',
 'learn',
 'realli',
 'want',
 'tell',
 'friend',
 'famili',
 'voic',
 'show',
 'casual',
 'interact',
 'technolog',
 'help',
 'none',
 'great',
 'product',
 'smart',
 'technolog',
 'sure',
 'say',
 'hey',
 'googl',
 'activ',
 'show',
 'enough',
 'googl',
 'talk',
 'ask',
 'question',
 'small',
 'fit',
 'home',
 'decor',
 'question',
 'ask',
 'googl',
 'nest',
 'sound',
 'bite',
 'stick',
 'mind',
 'hear',
 'ad',
 'search',
 'googl',
 'talk',
 'googl',
 'nest',
 'sure',
 'fact',
 'talk',
 'commun',
 'devic',
 'get',
 'respons',
 'connect',
 'internet',
 'voic',
 'okay',
 'honey',
 'cha',
 'coo',
 'love',
 'love',
 'comfort',
 'part',
 'home',
 'relax',
 'easi',
 'probabl',
 'soon',
 'enough',
 'good',
 'tal

In [133]:
test = word_tokens[:5]
test

[['that', 'you', 'can', 'talk', 'to', 'the', 'internet'],
 ['not', 'into', 'it'],
 ['apple'],
 ['it', "'s", 'to', 'make', 'everyday', 'life', 'easier', '.'],
 ['showing', 'me', 'the', 'product', 'and', 'how', 'it', 'works']]

In [85]:
word_tokens2 = []
for review in test:
    sublist = []
    for word in review:
        if word not in stop_words:
            nltk.pos_tag(nltk.word_tokenize(word)
            lemmatizer.lemmatize(word)
            sublist.append(word)
    word_tokens2.append(sublist)
        

In [86]:
word_tokens2

[['talk', 'internet'],
 [],
 ['apple'],
 ["'s", 'make', 'everyday', 'life', 'easier', '.'],
 ['showing', 'product', 'works']]

In [88]:
print(lemmatizer.lemmatize('shows'))

show


In [90]:
for review in test:
    print(review)
    #print(nltk.pos_tag(nltk.word_tokenize(review)))

['that', 'you', 'can', 'talk', 'to', 'the', 'internet']
['not', 'into', 'it']
['apple']
['it', "'s", 'to', 'make', 'everyday', 'life', 'easier', '.']
['showing', 'me', 'the', 'product', 'and', 'how', 'it', 'works']


In [91]:
meep = ['that', 'you', 'can', 'talk', 'to', 'the', 'internet']

In [94]:
print(nltk.pos_tag(nltk.word_tokenize("showing")))

[('showing', 'VBG')]


In [95]:
from nltk.corpus import wordnet

In [101]:
test_sentence = "That you can talking to the internet"

In [97]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [98]:
lemmatizer = WordNetLemmatizer()

In [99]:
word = 'feet'
print(lemmatizer.lemmatize(word, get_wordnet_pos(word)))

foot


In [102]:
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(test_sentence)])

['That', 'you', 'can', 'talk', 'to', 'the', 'internet']


In [112]:
for review in test:
    test = " ".join(review)
    print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(test)])

['that', 'you', 'can', 'talk', 'to', 'the', 'internet']
['not', 'into', 'it']
['apple']
['it', "'s", 'to', 'make', 'everyday', 'life', 'easy', '.']
['show', 'me', 'the', 'product', 'and', 'how', 'it', 'work']


In [121]:
symbols = "!\"“'’#$%&()*+-./:;<=>?@[\]^_`{|}~\n0123456789"
for i in symbols:
    for review in test:
        review = np.char.replace(review, i, "")

In [134]:
test

[['that', 'you', 'can', 'talk', 'to', 'the', 'internet'],
 ['not', 'into', 'it'],
 ['apple'],
 ['it', "'s", 'to', 'make', 'everyday', 'life', 'easier', '.'],
 ['showing', 'me', 'the', 'product', 'and', 'how', 'it', 'works']]

In [123]:
veep = ['it', "'s", 'to', 'make', 'everyday', 'life', 'easier', '.']

In [125]:
for i in veep:
    for a in symbols:
        if a in i:
            i.strip("!\"“'’#$%&()*+-./:;<=>?@[\]^_`{|}~\n0123456789")

In [127]:
veep = [i.strip("!\"“'’#$%&()*+-./:;<=>?@[\]^_`{|}~\n0123456789") for i in veep]
veep

['it', 's', 'to', 'make', 'everyday', 'life', 'easier', '']

In [128]:
veep = [word for word in veep if len(word) > 1]
veep

['it', 'to', 'make', 'everyday', 'life', 'easier']

In [132]:
test

'showing me the product and how it works'

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [135]:
cleaned_test = []
for review in test:
    test2 = " ".join(review)
    review = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(test2)]
    review = [i.strip("!\"“'’#$%&()*+-./:;<=>?@[\]^_`{|}~\n0123456789") for i in review]
    review = [word for word in review if len(word) > 1]
    cleaned_test.append(review)

In [136]:
cleaned_test

[['that', 'you', 'can', 'talk', 'to', 'the', 'internet'],
 ['not', 'into', 'it'],
 ['apple'],
 ['it', 'to', 'make', 'everyday', 'life', 'easy'],
 ['show', 'me', 'the', 'product', 'and', 'how', 'it', 'work']]

TypeError: 'type' object is not iterable